In [10]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
# @Time    : 2022/10/09 16:19
# @Author  : Wang Yujia
# @File    : SA_for_PT_model.ipynb
# @Description : Parameter estimation for PT_model using Simulated Annealing and Delta==1

# 0. What for

# 1. Preparations
1. infer参数一是需要data，二是需要把p表示出来才能写出来loss func
2. data来自`data_selected_path`

## 1.1 全局设置
1. 除了表示uniq auction的features，还引入了
    - 'cnt_uniq':表示paper里的Loss function公式里的A

In [4]:
# data path
# data_selected_path = "../data/info_asymm/datawithnp_asc_symmetry_2_selected.csv"
data_selected_path = "../data/SA_PT/datawithnp_GT_selected.csv"
# data_key path
data_key_path = "../data/SA_PT/data_key.csv"
# optimized parameters' saving path:
params_opitim_path = "../data/SA_PT/params_opitim.csv"
params_opitim_delta_path = "../data/SA_PT/params_opitim_delta_T.csv"
params_opitim_delta_wset_path = "../data/SA_PT/params_opitim_delta_T_wset.csv"

# features that GT need
features_GT = ['product_id','bidincrement','bidfee','retail']
features_GT_infer = ['cnt_uniq']

# for SA
# initial params
table_5_M = [0.025,3.72]
# table_5_M = [0,2]
# lower/ upper bound
lb = [-0.3,0.01]
ub = [0.3,18]

import numpy as np
import pandas as pd
import seaborn as sns
from SA_modified import SABoltzmann
from SA_for_PT_funcs_delta_eq1 import *
import matplotlib.pyplot as plt
import datetime
from sko.tools import set_run_mode
from visdom import Visdom

# viz = Visdom(env = "Loss of PT_2")

## 1.2 data 读取
1. 读取data以做SA
2. 提取出来`data_key`，以及其他计算需要的features

In [5]:
data = pd.read_csv(data_selected_path, encoding="utf-8")
data_key = data[features_GT].copy()
data_key.drop_duplicates(inplace=True,ignore_index=True)
data_key.to_csv(data_key_path,header=True, encoding="utf-8",index=False)

# 需要计算`N_uniq_auction`组setting下的结果
N_uniq_auction= data_key.shape[0]

print("For PT model, there are *{}* settings waiting to be inferred.".format(N_uniq_auction))

For PT model, there are *1303* settings waiting to be inferred.


## 1.3 functions about 'key'

In [6]:
# get key from i in 'data_key'
def get_key_from_index(i,str="NotStr"):
    if(str=="str"):
        key_i = list(data_key.iloc[i,:])
        key_i_str = (str(key_i[0]),str(key_i[1]),str(key_i[2]))
        return key_i_str
    else:
        key_i = data_key.iloc[i,:]
        return key_i

#features_GT = ['product_id','bidincrement','bidfee','retail']
def select_data_fromkey(key_i_str):
    return data[(data['product_id'] == key_i_str[0]) & (data['bidincrement'] == key_i_str[1]) & (data['bidfee'] == key_i_str[2]) & (data['retail'] == key_i_str[3])].copy()


# 2. Equi. condition in PT model
1. 根据Eq(6)
2. 注意分辨怎么代入上面的公式
3. `delta = 1`时，公式可以大大化简，见ipad上的公式

In [7]:
# for scipy use
def func_1(u,*args):
    alpha,delta,labda,t,b,tmp = args
    return (labda * f(x=C(t-1, b), alpha=alpha) - labda * OMEGA(u) * f(x=(C(t-1, b) + b), alpha=alpha) + OMEGA(1-u) * f(tmp, alpha))

# for scipy use
def func_2(u,*args):
    alpha,delta,labda,t,b,tmp = args
    return (-f(x=C(t-1, b), alpha=alpha) + OMEGA(u) * f(x=(C(t-1, b) + b), alpha=alpha) + (1 - OMEGA(u)) * f(-tmp, alpha))

def f_Equi(t,v,d,b,alpha,labda):

    tmp = v-d*t-C(t-1,b) - b

    if (tmp>=0):
        root = (labda*f(C(t-1,b),alpha) + f(tmp,alpha)) / (labda*f(C(t-1,b)+b,alpha) + f(tmp,alpha))
        # if(np.isclose(root,0.0)):
        #     print(f"t:{t} ---- u = 0.0:{root} ---- alpha : {alpha}")
    else:
        # print("tmp starts to < 0", t)
        root = (f(C(t-1,b),alpha) - f(-tmp,alpha)) / (f(C(t-1,b)+b,alpha) + f(-tmp,alpha))
        # if(np.isclose(root,0.0)):
        #     print(f"t:{t} -- u = 0.0:{root} -- alpha : {alpha} -- lamda : {labda}")

    # if(root > 1.0):
    #     print(f"t:{t} ---- u > 1.0:{root} ---- alpha: {alpha}")

    #viz.line([[0.0,0.0]],[0],win = 'root compare',opts= dict(title='root in 2 methods'+str(t),legend=['simplify', 'sympy']))
    #viz.line([[np.float(root1),np.float(root)]],[t],win = 'root compare', update='append')

    return root

# 3. SA
## 3.1 define loss function
1. loss function: NLL for auctions with same `features_GT`
2.

In [20]:
def loss_func(params,other_params):
    # start_time_loss = datetime.datetime.now()
    alpha = params[0]
    # delta = 1
    labda = params[1]
    max_T,v,d,b,T_i,cnt_n_2_i = other_params
    flag = 1
    # solve for U from Equi. condt.
    U_i = [0] * (max_T + 1)
    U_i[0] = 1

    for t in range(1,max_T+1):

        U_i[t] = f_Equi(t, v, d, b, alpha, labda)
        if(flag & (U_i[t]<=0)):
            # print(f"t:{t} -- u starts to <= 0.0: {U_i[t]} -- alpha : {alpha} -- lamda : {labda}")
            flag = 0

    # calculate NLL under this auction setting & PT params
    nll = 0.0
    NLL1 = 0.0
    if(U_i[0]==1):
        U_i.pop(0)
    for idx in range(0,data_i.shape[0]):
        # sum up the log prob among all durations of this auction
        # NLL1 += np.log(np.cumprod(U_i)[T_i[idx]-2]*(1-U_i[T_i[idx]-1]))* cnt_n_2_i[idx]
        # NLL1 == nll
        nll += ( np.sum( np.log( U_i[0:(T_i[idx]-1)] ) ) + np.log(1-U_i[(T_i[idx]-1)]) )* cnt_n_2_i[idx]
    # print('> The loss costs {time_costs}s \n'.format(time_costs=(datetime.datetime.now() - start_time_loss).total_seconds()))

    return float(-nll)

## 3.2 do SA
1. 要对每一个setting做一次infer == 对每一个setting执行一次SA。
    - 可以并行吗？YES
2. 具体的：对每个setting `i`
    - 每一个setting `i` 可以提取出来一个`data_i`，代表所有auction
    - 每一个`data_i`中的`cnt_uniq`，也就是`A`，是相同的，表示setting `i` 进行的拍卖总次数.【但是这个`A`在计算loss的时候派不上用场】
    - `N`表示duration，因此paper公式里的$T_a$即`N[a]`
    - 因此有`A = sum(data_i['cnt_n_2'])`，其中的'cnt_n_2'表示了该行对应的`duration=N`发生的次数
    - 按照上文，求解`U[i]_t` which is a array with shape of (max(N)),也就是求解paper里的`p_t`
3.每次进行`L`次对参数的试探寻找，每次寻找对应一个温度一组新的参数。
    - 优化的完成/退出条件：温度小于`T_min`或者最低温度保持`max_stay_counter`次的不变
    - 鉴于温度小于`T_min`很难达到，因此基本上对一组参数进行优化要进行L*max_stay_counter+1次运算（loss运算）

In [21]:
params_opitim = pd.DataFrame(columns=['key_idx','alpha','delta','labda','initial_loss','final_loss','avg_loss'])

In [22]:
# Perform SA respectively for all settings
# Fail: [30,124,151,175,235,331,416,417,524,546,572,821,940,941,1091]
# Fail 2.2:[1234]
for i in range(0,20):
    #start_time = datetime.datetime.now()

    # get i_th data_key
    key_i = get_key_from_index(i)
    # extract data with same `key_i` into a table
    data_i = select_data_fromkey(key_i)
    data_i.reset_index(drop=True,inplace=True)

    T_i = data_i['N'].astype(int)          # auction duration sequence
    max_T = int(max(T_i))                  # max duration value

    cnt_n_2_i = data_i['cnt_n_2'].astype(int)       # Number of occurrences of different durations
    # for a certain auction(like 'data_i'), 'cnt_uniq' should be all the same
    A_i = int(data_i['cnt_uniq'].unique())
    assert(A_i == sum(cnt_n_2_i),"'cnt_uniq' does not match with sum of 'cnt_n_2'!")

    v = float(data_i['retail'].unique())
    d = float(data_i['bidincrement'].unique())
    b = float(data_i['bidfee'].unique())
    assert(max_T <= (v-b)/d)

    # calculate NLL
    print(f"> For the *{i}_th* data_key")
    print("> retail = {0},bidincrement = {1}, bidfee = {2}, infer PT's parameters".format(v,d,b))
    print("> Initilizing SA....... \n")
    set_run_mode(loss_func, 'cached')
    set_run_mode(loss_func, 'multithreading')

    # L=30, max_stay_counter=15
    sa_boltzmann = SABoltzmann(func=loss_func, x0=table_5_M, other_params = [max_T,v,d,b,T_i,cnt_n_2_i],T_max=round((v-d)/b), T_min=1, learn_rate=0.2, L=15, max_stay_counter=5,
                            lb=lb, ub=ub)

    print("> Now do SA....... \n")
    best_x, best_y = sa_boltzmann.run()
    # print('> The whole inference process costs {time_costs}s \n'.format(time_costs=(datetime.datetime.now() - start_time).total_seconds()))

    print("> SA ENDS....... \n")
    # if(i%10 == 0):
    #     viz.line([0.0]*(sa_boltzmann.iter_cycle+1),[0]*(sa_boltzmann.iter_cycle+1),env = "Loss of PT_2",win = 'Loss of '+str(i),opts= dict(title='Loss of '+str(i)))
    #     viz.line(np.array(sa_boltzmann.generation_best_Y),np.arange(0,sa_boltzmann.iter_cycle+1),env = "Loss of PT_2",win = 'Loss of '+str(i), update='append')

    # append the opitimized params into the df
    df_tmp = pd.DataFrame([[i,best_x[0],1,best_x[1],sa_boltzmann.generation_best_Y[0],best_y,best_y/A_i]],columns=['key_idx','alpha','delta','labda','initial_loss','final_loss','avg_loss'])
    params_opitim = params_opitim.append(df_tmp,ignore_index=True)  # ignore_index=True could help in rearranging index

<>:19: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<>:19: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<>:19: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<ipython-input-22-39c674076801>:19: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(A_i == sum(cnt_n_2_i),"'cnt_uniq' does not match with sum of 'cnt_n_2'!")


> For the *0_th* data_key
> retail = 169.99,bidincrement = 0.15, bidfee = 0.75, infer PT's parameters
> Initilizing SA....... 

NLL: -766.4114056679603, NLL1: -766.4114056679604
> Now do SA....... 

NLL: -460.4356710495317, NLL1: -460.4356710495317
NLL: -241.6105260955904, NLL1: -241.6105260955904
NLL: -240.4355677526338, NLL1: -240.4355677526338
NLL: -241.83974201830833, NLL1: -241.83974201830833
NLL: -268.94128601945323, NLL1: -268.9412860194533
NLL: -325.18356611051087, NLL1: -325.18356611051087
NLL: -273.60306635568753, NLL1: -273.60306635568753
NLL: -263.2393140866859, NLL1: -263.2393140866859
NLL: -246.06492353022063, NLL1: -246.0649235302206
NLL: -268.63183079941587, NLL1: -268.63183079941587
NLL: -241.56778226842192, NLL1: -241.56778226842192
NLL: -264.70439758323306, NLL1: -264.70439758323306
NLL: -343.19125446228117, NLL1: -343.19125446228117
NLL: -362.84792094799104, NLL1: -362.84792094799116
NLL: -455.8969236555344, NLL1: -455.8969236555343
NLL: -348.78431933784697, NLL1: -

D:\Anaconda\envs\pythorch\lib\site-packages\ipykernel_launcher.py:26: RuntimeWarning: divide by zero encountered in log


NLL: -404.32780865865186, NLL1: -404.3278086586518
NLL: -322.94094272813516, NLL1: -322.9409427281351
NLL: -390.3604312144401, NLL1: -390.3604312144401
NLL: -355.86154228280054, NLL1: -355.8615422828005
NLL: -441.1665533434448, NLL1: -441.1665533434447
NLL: -221.45152836515817, NLL1: -221.45152836515823
NLL: -274.2765453891772, NLL1: -274.2765453891773
NLL: -254.90573933019817, NLL1: -254.90573933019817
NLL: -208.77511755733363, NLL1: -208.77511755733366
NLL: -228.21839562552773, NLL1: -228.21839562552773
NLL: -254.41382192932207, NLL1: -254.41382192932207
NLL: -317.4437578485473, NLL1: -317.4437578485473
NLL: -272.4153133921665, NLL1: -272.4153133921665
NLL: -272.2037228608963, NLL1: -272.20372286089633
NLL: -300.52665518962664, NLL1: -300.52665518962664
NLL: -260.32328070364747, NLL1: -260.32328070364747
NLL: -262.19959276804127, NLL1: -262.19959276804127
NLL: -588.1208206219646, NLL1: -588.1208206219646
NLL: -448.8996090487474, NLL1: -448.8996090487474
NLL: -642.8755610950232, NLL1:

D:\Anaconda\envs\pythorch\lib\site-packages\ipykernel_launcher.py:27: RuntimeWarning: divide by zero encountered in log


...

D:\Anaconda\envs\pythorch\lib\site-packages\ipykernel_launcher.py:26: RuntimeWarning: divide by zero encountered in log
D:\Anaconda\envs\pythorch\lib\site-packages\ipykernel_launcher.py:27: RuntimeWarning: divide by zero encountered in log


NLL: -219.35502413096057, NLL1: -219.35502413096054
NLL: -445.8072605820809, NLL1: -445.80726058208086
NLL: -519.766338964832, NLL1: -519.766338964832
NLL: -274.7250352201861, NLL1: -274.72503522018616
NLL: -291.003941022982, NLL1: -291.00394102298196
NLL: -322.40808724655074, NLL1: -322.40808724655074
NLL: -339.1733162702639, NLL1: -339.1733162702639
NLL: -326.79859982975114, NLL1: -326.7985998297511
NLL: -408.6629290150284, NLL1: -408.6629290150284
NLL: -285.0874224946134, NLL1: -285.0874224946134
NLL: -inf, NLL1: -inf
NLL: -inf, NLL1: -inf
NLL: -650.474767286825, NLL1: -650.474767286825
NLL: -381.23493456149833, NLL1: -381.23493456149833
NLL: -302.1546474337817, NLL1: -302.1546474337817
NLL: -323.86670272009445, NLL1: -323.8667027200945
NLL: -762.9455657479683, NLL1: -762.9455657479683
NLL: -223.03720534497344, NLL1: -223.03720534497344
NLL: -222.1566166556352, NLL1: -222.1566166556352
NLL: -341.0578418675677, NLL1: -341.0578418675677
NLL: -568.2756960269352, NLL1: -568.275696026935

D:\Anaconda\envs\pythorch\lib\site-packages\ipykernel_launcher.py:26: RuntimeWarning: divide by zero encountered in log
D:\Anaconda\envs\pythorch\lib\site-packages\ipykernel_launcher.py:27: RuntimeWarning: divide by zero encountered in log


NLL: -1494.4859805666529, NLL1: -1494.4859805666533
NLL: -4192.554725849394, NLL1: -inf
NLL: -972.5169550126768, NLL1: -972.5169550126768
NLL: -inf, NLL1: -inf
NLL: -757.7929402705399, NLL1: -757.7929402705399
NLL: -740.0233082136821, NLL1: -740.0233082136821
NLL: -1048.2167414813973, NLL1: -1048.2167414813973
NLL: -749.8568605687406, NLL1: -749.8568605687407
NLL: -1311.8576006861072, NLL1: -1311.8576006861072
NLL: -inf, NLL1: -inf
NLL: -1207.8237559938395, NLL1: -1207.8237559938395
NLL: -1489.4946940352538, NLL1: -1489.4946940352538
NLL: -1345.6493861127137, NLL1: -1345.6493861127137
NLL: -810.026874232943, NLL1: -810.026874232943
NLL: -1206.9307004553373, NLL1: -1206.930700455337
NLL: -818.6950949843225, NLL1: -818.6950949843226
NLL: -948.1767662532825, NLL1: -948.1767662532825
NLL: -inf, NLL1: -inf
NLL: -1193.3115504273633, NLL1: -1193.3115504273633
NLL: -882.2706593769905, NLL1: -882.2706593769905
NLL: -749.6800813465295, NLL1: -749.6800813465295
NLL: -5498.459532026326, NLL1: -inf

KeyboardInterrupt: 

In [38]:
# save 'params_opitim' for later check
params_opitim.to_csv(params_opitim_path, header=True, encoding="utf-8",index=False)

# 4. append两个表
1. 执行1次

In [40]:
params_opitim = pd.read_csv(params_opitim_delta_path, encoding="utf-8")
assert(params_opitim.shape[0] == data_key.shape[0])
params_opitim_withsetting = pd.concat([data_key,params_opitim],axis = 1)

In [28]:
max_T_tmp = []
for i in range(0,N_uniq_auction):

    # get optimized params
    alpha, labda, delta = params_opitim.iloc[i][0],params_opitim.iloc[i][1],params_opitim.iloc[i][2]

    # get i_th data_key
    key_i = get_key_from_index(i)
    # extract data with same `key_i` into a table
    data_i = select_data_fromkey(key_i)
    data_i.reset_index(drop=True,inplace=True)

    T_i = data_i['N'].astype(int)          # auction duration sequence
    max_T = int(max(T_i))                  # max duration value
    max_T_tmp.append(max_T)

params_opitim_withsetting['max_T'] = np.array(max_T_tmp)
params_opitim_withsetting['T'] = (params_opitim_withsetting.retail - params_opitim_withsetting.bidfee) / params_opitim_withsetting.bidincrement
params_opitim_withsetting.head(5)

params_opitim_withsetting.to_csv(params_opitim_delta_wset_path, header=True, encoding="utf-8",index=False)

# 5. TEST
1. 对之前infer不出来的一些setting拿出来，看看在一组固定的参数下，u的变化
    - 其实当max_T过大的时候，u会变成负数

In [41]:
viz = Visdom(env='001')
# 改这个pls
i=9
# 参数固定下
alpha = 0.002
labda = 0.38

key_i = get_key_from_index(i)
data_i = select_data_fromkey(key_i)
data_i.reset_index(drop=True,inplace=True)

T_i = data_i['N'].astype(int)
v = float(data_i['retail'].unique())
d = float(data_i['bidincrement'].unique())
b = float(data_i['bidfee'].unique())
max_T = int(max(T_i))                  # max duration value
U_i = [0] * (max_T + 1)
U_i[0] = 1

for t in range(1,max_T+1):

    U_i[t] = f_Equi(t, v, d, b, alpha, labda)

viz.line([0.0]*(max_T+1),[0]*(max_T+1),win = 'Loss of '+str(i),env='001',opts= dict(title='Loss of '+str(i)))
viz.line(np.array(U_i),np.arange(0,max_T+1),win = 'Loss of '+str(i), env='001',update='append')

Setting up a new session...


tmp starts to < 0 2498
tmp starts to < 0 2499
tmp starts to < 0 2500
tmp starts to < 0 2501
tmp starts to < 0 2502
tmp starts to < 0 2503
tmp starts to < 0 2504
tmp starts to < 0 2505
tmp starts to < 0 2506
tmp starts to < 0 2507
tmp starts to < 0 2508
tmp starts to < 0 2509
tmp starts to < 0 2510
tmp starts to < 0 2511
tmp starts to < 0 2512
tmp starts to < 0 2513
tmp starts to < 0 2514
tmp starts to < 0 2515
tmp starts to < 0 2516
tmp starts to < 0 2517
tmp starts to < 0 2518
tmp starts to < 0 2519
tmp starts to < 0 2520
tmp starts to < 0 2521
tmp starts to < 0 2522
tmp starts to < 0 2523
tmp starts to < 0 2524
tmp starts to < 0 2525
tmp starts to < 0 2526
tmp starts to < 0 2527
tmp starts to < 0 2528
tmp starts to < 0 2529
tmp starts to < 0 2530
tmp starts to < 0 2531
tmp starts to < 0 2532
tmp starts to < 0 2533
tmp starts to < 0 2534
tmp starts to < 0 2535
tmp starts to < 0 2536
tmp starts to < 0 2537
tmp starts to < 0 2538
tmp starts to < 0 2539
tmp starts to < 0 2540
tmp starts 

'Loss of 9'

In [31]:
#print(viz.get_env_list())

19.8

In [21]:

# 保存整个环境
viz.save(envs=['Loss of PT'])

'["Loss of PT"]'